In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [2]:
df = pd.read_csv("data/data.csv")
test_text = df["text"][1]

def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature_spec)).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({"signature_name": "serving_default", "instances": result})

In [3]:
import requests

def make_predictions(text, model_url="http://localhost:8080/v1/models/fakenews-detection-model:predict", threshold=0.6):

  try:
    json_data = prepare_json(text)

    response = requests.post(model_url, data=json_data)
    response.raise_for_status()  # Raise an exception for non-200 status codes

    prediction = response.json()["predictions"][0][0]
    return "Real News!" if prediction > threshold else "Fake News!"

  except requests.exceptions.RequestException as e:
    print(f"An error occurred while making the prediction: {e}")
    return "Prediction Error: Check network connection or model availability."

In [4]:
make_predictions(test_text)

'Real News!'

In [5]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/fakenews-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1721465839'}]}
